In [1]:
in_path = '../data/raw/Data OPCS/1181.xls'
out_path = '../data/interim/OPCS_master.pkl'

In [2]:
from os.path import dirname
import os, sys, inspect

currentdir = os.getcwd()
parentdir = dirname(currentdir)

sys.path.insert(0,parentdir)

# Libs

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import pickle

from src.utils import get_name_of_semester, read_pickle, dump_to_pickle

In [4]:
pd.set_option('display.max_rows', None)

# Import Table

In [5]:
cht_18_raw = pd.read_excel(in_path,
                          usecols=[1,4,5,6,7,8,9,10],
                          converters={'ACAD_PROG':str,
                                     'EMPLID':str,
                                     'CAMPUS_ID':str})

In [6]:
cht_18_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10454 entries, 0 to 10453
Data columns (total 8 columns):
EMPLID        10454 non-null object
ACAD_PROG     10454 non-null object
ADMIT_TERM    10454 non-null int64
STRM          10368 non-null float64
CUR_GPA       10368 non-null float64
CUM_GPA       10368 non-null float64
DESCR         10454 non-null object
CAMPUS_ID     10454 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 653.5+ KB


In [7]:
cht_18_raw.head(8)

,EMPLID,ACAD_PROG,ADMIT_TERM,STRM,CUR_GPA,CUM_GPA,DESCR,CAMPUS_ID
0,01011180417,54541,1181,NaN,NaN,NaN,Pengelolaan Perhotelan,1811401617
1,01011180430,54541,1181,NaN,NaN,NaN,Pengelolaan Perhotelan,1811404439
2,01051180135,54541,1181,NaN,NaN,NaN,Pengelolaan Perhotelan,1811402802
3,01051180135,54541,1181,NaN,NaN,NaN,Pengelolaan Perhotelan,1811402802
4,01121190090,54541,1181,NaN,NaN,NaN,Pengelolaan Perhotelan,1811404507
5,01541180001,54541,1181,1181.0,3.50,3.50,Pengelolaan Perhotelan,1831100345
6,01541180001,54541,1181,1182.0,3.06,3.28,Pengelolaan Perhotelan,1831100345
7,01541180001,54541,1181,1183.0,3.58,3.33,Pengelolaan Perhotelan,1831100345


In [8]:
cht_18_raw.shape

(10454, 8)

In [9]:
cht_18_raw.isnull().sum()

EMPLID         0
ACAD_PROG      0
ADMIT_TERM     0
STRM          86
CUR_GPA       86
CUM_GPA       86
DESCR          0
CAMPUS_ID      0
dtype: int64

# Cleaning

In [10]:
#rearrange columns
arr_cols = [
    'EMPLID',
    'CAMPUS_ID',
    'ACAD_PROG',
    'DESCR',
    'ADMIT_TERM',
    'STRM',
    'CUR_GPA',
    'CUM_GPA'
]

cht_18_raw = cht_18_raw[arr_cols]

In [11]:
#rename columns
cht_18_master = cht_18_raw.rename(columns={
    'EMPLID': 'NIM',
    'ACAD_PROG': 'major_code_opcs',
    'CAMPUS_ID': 'form_number',
    'DESCR': 'major_name_opcs',
    'CUR_GPA': 'GPA',
    'CUM_GPA': 'CGPA'
})

In [12]:
#drop missing values
cht_18_master.dropna(inplace=True)

In [13]:
#format STRM type to int
cht_18_master['STRM'] = cht_18_master['STRM'].astype(int)

In [14]:
cht_18_master.head()

,NIM,form_number,major_code_opcs,major_name_opcs,ADMIT_TERM,STRM,GPA,CGPA
5,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1181,3.50,3.50
6,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1182,3.06,3.28
7,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1183,3.58,3.33
8,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1191,0.00,3.33
9,01541180002,1811400700,54541,Pengelolaan Perhotelan,1181,1181,0.00,0.00


### Get Normalized Semester Name

In [15]:
cht_18_master['semester'] = \
cht_18_master.apply(lambda row: get_name_of_semester(row['ADMIT_TERM'], row['STRM']),
                   axis=1)

In [16]:
#drop sem_04 since the semester is not finished yet
#drop sem_99, nonsense semester 
semester_to_drop =['sem_04', 'sem_99']

cht_18_master = cht_18_master.loc[~cht_18_master.semester.isin(semester_to_drop)]

In [17]:
cht_18_master.head()

,NIM,form_number,major_code_opcs,major_name_opcs,ADMIT_TERM,STRM,GPA,CGPA,semester
5,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1181,3.50,3.50,sem_01
6,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1182,3.06,3.28,sem_02
7,01541180001,1831100345,54541,Pengelolaan Perhotelan,1181,1183,3.58,3.33,sem_03
9,01541180002,1811400700,54541,Pengelolaan Perhotelan,1181,1181,0.00,0.00,sem_01
10,01541180002,1811400700,54541,Pengelolaan Perhotelan,1181,1182,0.00,0.00,sem_02


In [18]:
#drop changed major data, keep the updated major data
cht_18_master = cht_18_master.loc[cht_18_master['CGPA']!=0]

### Get Last Semester CGPA

In [19]:
df = cht_18_master.sort_values(['NIM', 'STRM'], ascending=True)

In [20]:
df['lead_sem'] = df.groupby('NIM')['semester'].shift(-1)

In [21]:
df['is_lead_sem_null'] = df['lead_sem'].isnull()

In [22]:
df_last_sem = df[df['is_lead_sem_null']==True][['NIM', 'form_number', 'major_code_opcs', 'major_name_opcs', 'CGPA']]

In [23]:
df_last_sem.rename(columns={
    'CGPA': 'Last_CGPA'
}, inplace=True)

# Pivot

In [24]:
df_CGPA = \
df.pivot_table(index=['NIM', 'form_number', 'major_code_opcs', 'major_name_opcs'], columns='semester', values='CGPA').reset_index()

df_GPA = \
df.pivot_table(index=['NIM', 'form_number', 'major_code_opcs', 'major_name_opcs'], 
               columns='semester', values='GPA').reset_index()

In [25]:
OPCS_master = pd.merge(df_CGPA, df_GPA, on=['NIM','form_number', 'major_code_opcs', 'major_name_opcs'], suffixes=('_CGPA', '_GPA'))

In [26]:
OPCS_master = OPCS_master.merge(df_last_sem, on=['NIM','form_number','major_code_opcs','major_name_opcs'])

In [27]:
OPCS_master.head()

,NIM,form_number,major_code_opcs,major_name_opcs,sem_01_CGPA,sem_02_CGPA,sem_03_CGPA,sem_01_GPA,sem_02_GPA,sem_03_GPA,Last_CGPA
0,01011180001,1831100382,01011,Manajemen,3.54,3.30,3.25,3.54,3.04,2.98,3.25
1,01011180002,1811100709,01011,Manajemen,1.06,1.06,1.06,1.06,0.00,0.00,1.06
2,01011180003,1811100388,01011,Manajemen,2.39,2.03,2.07,2.39,1.63,2.25,2.07
3,01011180004,1831100016,01011,Manajemen,3.08,2.99,2.91,3.08,2.89,2.56,2.91
4,01011180005,1811100684,01011,Manajemen,3.54,3.27,3.28,3.54,2.98,3.30,3.28


In [28]:
OPCS_master.shape

(2571, 11)

# Dump to Pickle

In [29]:
dump_to_pickle(OPCS_master, out_path)